In [1]:
#-------------Imports--------------
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

In [2]:
#-----------Load Dataset-----------

path_train = "fashion_train.npy"
path_test = "fashion_test.npy"

train = np.load(path_train)
test = np.load(path_test)

#Split the training and test data into features and labels
X = train[:,:784]
y = train[:,784]

X = X / 255

X_test = test[:,:784]
y_test = test[:,784]

X_test = X_test / 255

print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

(10000, 784)
(10000,)
(5000, 784)
(5000,)


### Split training set into training and validation set

In [3]:
train_percentage = int((X.shape[0]) * 0.7)
train_percentage

X_train = X[:train_percentage, :]
y_train = y[:train_percentage]

print(X_train.shape)
print(y_train.shape)

X_val = X[train_percentage:, :]
y_val = y[train_percentage:]

print(X_val.shape)
print(y_val.shape)

(7000, 784)
(7000,)
(3000, 784)
(3000,)


### Final chosen model

In [4]:
model = Sequential([
    Dense(units=100, activation="relu"),
    Dropout(0.2),
    Dense(units=50, activation="relu"),
    Dropout(0.2),
    Dense(units=25, activation="relu"),
    Dropout(0.2),
    Dense(units=5, activation="linear")
])

model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer=Adam(learning_rate=1e-3),
              metrics=["accuracy"]
             )

model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))

Epoch 1/100
219/219 [==============================] - 2s 4ms/step - loss: 0.9061 - accuracy: 0.6304 - val_loss: 0.5642 - val_accuracy: 0.7947
Epoch 2/100
219/219 [==============================] - 1s 3ms/step - loss: 0.5948 - accuracy: 0.7817 - val_loss: 0.5003 - val_accuracy: 0.8250
Epoch 3/100
219/219 [==============================] - 1s 3ms/step - loss: 0.5301 - accuracy: 0.8043 - val_loss: 0.4399 - val_accuracy: 0.8363
Epoch 4/100
219/219 [==============================] - 1s 3ms/step - loss: 0.4776 - accuracy: 0.8257 - val_loss: 0.4141 - val_accuracy: 0.8417
Epoch 5/100
219/219 [==============================] - 1s 2ms/step - loss: 0.4458 - accuracy: 0.8323 - val_loss: 0.4202 - val_accuracy: 0.8333
Epoch 6/100
219/219 [==============================] - 1s 2ms/step - loss: 0.4383 - accuracy: 0.8381 - val_loss: 0.4074 - val_accuracy: 0.8497
Epoch 7/100
219/219 [==============================] - 1s 2ms/step - loss: 0.4186 - accuracy: 0.8499 - val_loss: 0.4020 - val_accuracy: 0.8413

### Make predictions and test accuracy on training set

In [5]:
logits_train2 = model(X_train)
probs_train2 = tf.nn.softmax(logits_train2)
probs_train2 = tf.make_tensor_proto(probs_train2)
probs_np2 = tf.make_ndarray(probs_train2)
y_pred2 = np.argmax(probs_np2, axis=1)
accuracy_score(y_pred2, y_train)

0.964

### Make predictions and test accuracy on validation set

In [6]:
logits_val2 = model(X_val)
probs_val2 = tf.nn.softmax(logits_val2)
probs_val2 = tf.make_tensor_proto(probs_val2)
probs_np_val2 = tf.make_ndarray(probs_val2)
y_pred_val2 = np.argmax(probs_np_val2, axis=1)
accuracy_score(y_pred_val2, y_val)

0.87

### Final test on the test set

In [7]:
logits_test2 = model(X_test)
probs_test2 = tf.nn.softmax(logits_test2)
probs_test2 = tf.make_tensor_proto(probs_test2)
probs_np_test2 = tf.make_ndarray(probs_test2)
y_pred_test2 = np.argmax(probs_np_test2, axis=1)
accuracy_score(y_pred_test2, y_test)

0.8546